https://www.bilibili.com/video/BV1vc411J7F4/?spm_id_from=333.337.search-card.all.click&vd_source=81884c519d60bbdad4b6fd87d340415f

In [1]:
!pip install langchain openai transformers python-dotenv
# !sudo apt-get install python3.9
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.3/205.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the

* update-alternatives: This is a Linux command used to maintain symbolic links determining default commands.
* –install: This option is used to install a new alternative for a particular command.
* /usr/bin/python3: This specifies the command for which you’re providing an alternative, which is `python3`.
* python3.7: This is the alternative command you want to set as the default.
* “1”: This is a priority value. When multiple alternatives exist, the one with the highest priority becomes the default.

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents import initialize
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [4]:
_ = load_dotenv(find_dotenv())
API_KEY = os.environ['OPENAI_API_KEY']

In [5]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

image_to_text_model = "Salesforce/blip-image-captioning-large"
processor = BlipProcessor.from_pretrained(image_to_text_model)
model = BlipForConditionalGeneration.from_pretrained(image_to_text_model).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [7]:
from transformers.models.oneformer.modeling_oneformer import OneFormerModelOutput
import requests
from PIL import Image

def describeImage(image_url):
    image_object = Image.open(requests.get(image_url, stream=True).raw).convert("RGB")
    # image
    inputs = processor(image_object, return_tensors='pt').to(device)
    outputs = model.generate(**inputs)
    return processor.decode(outputs[0], skip_special_tokens=True)

In [33]:
image_url = "https://www.rollingstone.com/wp-content/uploads/2018/06/trolls-trl_sq1975_s70_sq8011_s61_4k_rgb_fin_rgb-da9984e9-0429-41ad-9a9b-edc5f5855c8b.jpg?w=1581&h=1054&crop=1"
description = describeImage(image_url)
description

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'trolly and friends in a cartoon movie'

In [9]:
from langchain.tools import BaseTool

class DescribeImageTool(BaseTool):
    name = "Describe Image Tool"
    description = "use this tool to describe an image."

    def _run(self, url:str):
        description = describeImage(url)
        return description

    # Async operation
    def _arun(self, query: str):
        raise NotImplementedError("Async operation not supported yet")

tools = [DescribeImageTool()]

In [30]:
llm = ChatOpenAI(openai_api_key=API_KEY, temperature=0, model_name='gpt-3.5-turbo')
# chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True)
#memory = ConversationBufferWindowMemory(k=5, return_messages=True)

In [31]:
agent = initialize.initialize_agent(
    agent='chat-conversational-react-description',
    tools = tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

In [34]:
agent(f"Describe the following image:\n{image_url}")



> Entering new AgentExecutor chain...
{
    "action": "Describe Image Tool",
    "action_input": "https://www.rollingstone.com/wp-content/uploads/2018/06/trolls-trl_sq1975_s70_sq8011_s61_4k_rgb_fin_rgb-da9984e9-0429-41ad-9a9b-edc5f5855c8b.jpg?w=1581&h=1054&crop=1"
}

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



Observation: trolly and friends in a cartoon movie
Thought:{
    "action": "Final Answer",
    "action_input": "The image appears to be a scene from the animated movie 'Trolls', featuring the characters from the film."
}

> Finished chain.


{'input': 'Describe the following image:\nhttps://www.rollingstone.com/wp-content/uploads/2018/06/trolls-trl_sq1975_s70_sq8011_s61_4k_rgb_fin_rgb-da9984e9-0429-41ad-9a9b-edc5f5855c8b.jpg?w=1581&h=1054&crop=1',
 'chat_history': [HumanMessage(content='Describe the following image:\nhttps://1000logos.net/wp-content/uploads/2020/05/Google-Photos-logo.png'),
  AIMessage(content='The image is the Google Photos logo, which features four different colors.')],
 'output': "The image appears to be a scene from the animated movie 'Trolls', featuring the characters from the film."}

In [35]:
agent.memory.buffer

[HumanMessage(content='Describe the following image:\nhttps://1000logos.net/wp-content/uploads/2020/05/Google-Photos-logo.png'),
 AIMessage(content='The image is the Google Photos logo, which features four different colors.'),
 HumanMessage(content='Describe the following image:\nhttps://www.rollingstone.com/wp-content/uploads/2018/06/trolls-trl_sq1975_s70_sq8011_s61_4k_rgb_fin_rgb-da9984e9-0429-41ad-9a9b-edc5f5855c8b.jpg?w=1581&h=1054&crop=1'),
 AIMessage(content="The image appears to be a scene from the animated movie 'Trolls', featuring the characters from the film.")]